## Mace MD Demo notebook

This should demonstrate the bare bones of the API for setting up a variety of MD jobs with hybrid MACE-classical hamiltonians.  The same functionalty is exposed through the `mace-md` command line program that is installed with the `mace-openmm-interop` package


Your vs code server instance should have an attached CUDA GPU for this to run!

In [1]:
!nvidia-smi

Wed Jan  4 12:25:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   33C    P0    78W / 500W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from mace_openmm.mixed_system import MixedSystem
import torch
import logging
from typing import Optional, Union
from mace import tools

/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:88: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return '/'.join([path_part.strip('/ ') for path_part in split_path if path_part is not ''])
/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:703: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._output_mode is 'a':


In [3]:
logger = logging.getLogger("DEBUG")
tools.setup_logger(level="DEBUG", directory="./logs")

In [8]:
# Change inputs in this cell only
file = "../tests/example_data/butane.sdf"  # path to the system to be simulated, can be either protein ligand complex or isolated (sdf)
ml_mol = "CCCC"  # path to the proper topology of the small molecule to be modeleld by mace, since pdb files are not sufficiently detailed.  this should be sdf.
model_path = "../tests/example_data/MACE_SPICE_larger.model"
forcefields = [
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber14/DNA.OL15.xml",
]
# name of the residue to be parametrised by mace, should correspond to whatever it is called in the ml_mol sdf file and system file
resname = "butane"
# minimum separation between the solute and box wall
padding = 1.2
# ion concentration in the solute
ionicStrength = 0.15
# cutoff distance (nm) for the non-bonded terns of the classical forcefield
nonbondedCutoff = 1.0
# name of the neural network potential, should correpond to the implemented FFs
# in the openmm-ml package
potential = "mace"
# simulation temperature
temperature = 298.15
# precision to use for both MACE and openMM
dtype = torch.float64
# which version of the torch_nl to use - the n^2 impl is more reliable
neighbour_list = "torch_nl_n2"
# directory where simulation output will be written
output_dir = "./output"
# specify the type of system to create - pure (just the solute simulated in vcuum)
# hybrid (small molecule modelled by MACE, rest of the system (protein or solvent) modelled by classical forcefield), decoupled (for ABFE simulations - lambda parameter controls switching off the ligand non-bonded terms)
system_type = "pure"

In [9]:
torch.set_default_dtype(dtype)

In [10]:
mixedSystem = MixedSystem(
    file=file,
    ml_mol=ml_mol,
    model_path=model_path,
    forcefields=forcefields,
    resname=resname,
    padding=padding,
    ionicStrength=ionicStrength,
    nonbondedCutoff=nonbondedCutoff,
    potential=potential,
    temperature=temperature,
    dtype=dtype,
    neighbour_list=neighbour_list,
    output_dir=output_dir,
    system_type=system_type,
)

2023-01-04 12:26:18.907 DEBUG: OpenMM will use Double precision
2023-01-04 12:26:18.939 INFO: Using openff-1.0 unconstrained forcefield
Initialized topology with (14, 3) positions
MACE model compiled


In [12]:
# Once the mixed system is created, we can run several different types of simulation: 

mixedSystem.run_mixed_md(steps=1000, interval=100, output_file="./output_md_test.pdb")

In [13]:
# runs Markov chain monte carlo replica exchange to interpoolate between MM and MM/ML descriptions of the system
mixedSystem.run_replex_equilibrium_fep(replicas=2, restart=False, steps=2)